In [ ]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.pytorch
import warnings
warnings.filterwarnings('ignore')

# MLflow setup
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment('Transformer_TimeSeries_Example')

# Load data
df = pd.read_parquet('../data/sequences.parquet')
df['TIME'] = pd.to_datetime(df['TIME'])
df.set_index('TIME', inplace=True)

# Parameters
SEQUENCE_ID = 1  # Example with one sequence
TRAIN_SIZE = 100  # Adjusted to fit the dataset
VAL_SIZE = 28     # Adjusted to fit the dataset
TEST_SIZE = 28    # Adjusted to fit the dataset
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# Cell 2: Define Transformer Model
class TransformerTimeSeries(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim, dropout=0.1):
        super(TransformerTimeSeries, self).__init__()
        self.model_dim = model_dim
        
        # Embedding layer for input features
        self.embedding = nn.Linear(input_dim, model_dim)
        
        # Positional encoding
        self.positional_encoding = nn.Parameter(torch.zeros(1, TRAIN_SIZE, model_dim))
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output layer
        self.fc = nn.Linear(model_dim, output_dim)
    
    def forward(self, x):
        # Add positional encoding
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        
        # Pass through transformer encoder
        x = self.transformer_encoder(x)
        
        # Take the output of the last time step
        x = self.fc(x[:, -1, :])
        return x

# Cell 3: Preprocessing function
def preprocess_data(data):
    """Preprocess sequence data"""
    # Remove outliers
    Q1 = data['Power'].quantile(0.25)
    Q3 = data['Power'].quantile(0.75)
    IQR = Q3 - Q1
    data = data[
        (data['Power'] >= Q1 - 1.5 * IQR) &
        (data['Power'] <= Q3 + 1.5 * IQR)
    ].copy()
    
    # Scale data
    scaler = StandardScaler()
    data['Power_scaled'] = scaler.fit_transform(data[['Power']])
    
    return data, scaler

# Get sequence data
sequence_data = df[df['sequence'] == SEQUENCE_ID].copy()
sequence_data, scaler = preprocess_data(sequence_data)

# Use only half of the dataset
sequence_data = sequence_data.iloc[:len(sequence_data)//2]

# Split data
train_data = sequence_data['Power_scaled'][:TRAIN_SIZE].values
val_data = sequence_data['Power_scaled'][TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE].values
test_data = sequence_data['Power_scaled'][TRAIN_SIZE+VAL_SIZE:TRAIN_SIZE+VAL_SIZE+TEST_SIZE].values

# Prepare sequences for training
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 24  # Use 24 time steps as input
X_train, y_train = create_sequences(train_data, seq_length)
X_val, y_val = create_sequences(val_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Cell 4: Training and evaluation with Transformer
model = TransformerTimeSeries(
    input_dim=1,  # Input dimension (univariate time series)
    model_dim=64,  # Model dimension
    num_heads=4,  # Number of attention heads
    num_layers=2,  # Number of transformer layers
    output_dim=1  # Output dimension (univariate time series)
)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

# Start MLflow run
with mlflow.start_run(run_name=f"sequence_{SEQUENCE_ID}_transformer"):
    try:
        # Log parameters
        mlflow.log_params({
            'sequence': SEQUENCE_ID,
            'model_name': 'transformer',
            'train_size': TRAIN_SIZE,
            'val_size': VAL_SIZE,
            'test_size': TEST_SIZE,
            'batch_size': BATCH_SIZE,
            'epochs': EPOCHS,
            'learning_rate': LEARNING_RATE,
            'seq_length': seq_length,
            'model_dim': 64,
            'num_heads': 4,
            'num_layers': 2
        })
        
        # Training loop
        for epoch in range(EPOCHS):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(X_batch.unsqueeze(-1))
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()
            
            # Validation
            model.eval()
            with torch.no_grad():
                val_pred = model(X_val.unsqueeze(-1))
                val_loss = criterion(val_pred, y_val)
            
            print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")
        
        # Test predictions
        with torch.no_grad():
            test_pred = model(X_test.unsqueeze(-1))
            test_pred = scaler.inverse_transform(test_pred.numpy())
            test_data_actual = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))
        
        # Calculate metrics
        test_rmse = np.sqrt(mean_squared_error(test_data_actual, test_pred))
        test_mae = mean_absolute_error(test_data_actual, test_pred)
        
        # Log metrics to MLflow
        mlflow.log_metrics({
            'test_rmse': test_rmse,
            'test_mae': test_mae
        })
        
        # Print results
        print(f"\nResults for Transformer:")
        print(f"Test RMSE: {test_rmse:.4f}")
        print(f"Test MAE: {test_mae:.4f}")
        
    except Exception as e:
        print(f"Error: {str(e)}")